# Step8, 재귀에서 반복문으로 


앞 단계에서 Variable 클래스에 backward 메서드를 추가했었다.  
이번에는 처리 효율을 개선하고 앞으로의 확장을 대비해 backward 메서드의 구현 방식을 바꿔본다.

## 8.1 현재의 Variable 클래스 
이전 Variable 구조는 재귀 구조

## 8.2 반복문을 이용한 구현
이번에는 '재귀를 사용한 구현'을 '반복문을 이용한 구현'으로 고친다.

In [41]:
class Variable:
    def __init__(self, data):
        self.data = data 
        self.grad = None 
        self.creator = None 
    
    def set_creator(self, func):
        self.creator = func 
    
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()             # 함수를 가져온다. 
            x, y = f.input, f.output    # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다. 

            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가한다.

## 8.3 동작 확인 

In [42]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [43]:
# 함수 가져오기
A = Square()
B = Exp() 
C = Square() 

# 데이터 정의 및 순전파
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

# 역전파 
y.grad = np.array(1.0)
y.backward()
print(x.grad)


1.648721270700128
3.297442541400256


재귀에서 반복문으로 구현 방식을 전환한 이유

복잡한 계산 그래프 다룰수 있음, 처리효율 뛰어남 

**Caution_** 재귀는 함수를 재귀적으로 호출할 때마다 중간 결과를 메모리에 유지하면서(스텍에 쌓으면서) 처리를 이어가기 때문에 일반적으로 반복문 방식이 효율이 더 좋은 이유